In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
BASE_URL = "https://www.vinmec.com/vi/tin-tuc/hoi-dap-bac-si/?page={}"

In [4]:
page = 1
list_url = []
while page <= 563:
    url = BASE_URL.format(page)
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    h2_tags = soup.find_all('h2')

    for h2 in h2_tags:
        a_tags = h2.find_all('a')
        for a in a_tags:
            list_url.append(a['href'])
    
    page += 1

In [5]:
print(list_url[100])

/tin-tuc/hoi-dap-bac-si/buou-co-lanh-tinh-keo-dai-co-can-dieu-tri-khong/


In [45]:
BASE_URL_NEXT = "https://www.vinmec.com/vi"
qna_data = []

for url in list_url:
    response = requests.get(BASE_URL_NEXT + url)
    soup = BeautifulSoup(response.text, 'html5lib')

    div = soup.find_all("div", class_="rich-text")[0]
    ques = ""
    ans = ""
    is_question = False
    is_answer = False
    for i in div.children:
        if i.name:
            t = i.get_text().strip()
            if t == "Hỏi":
                is_question = True
            elif t == "Trả lời":
                is_question = False
                is_answer = True

            if is_question:
                ques += t + " "
            elif is_answer:
                ans += t + " "

        ques = ques.strip().replace("Hỏi", "", 1).strip()
        ans = ans.strip().replace("Trả lời", "", 1).strip()

    qna_data.append({
        'question': ques,
        'answer': ans
    })


In [46]:
df = pd.DataFrame(qna_data)
df.to_csv('vinmec_qna.csv', index=False, encoding='utf-8-sig')